In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import Adam, RMSprop, Nadam, AdaDelta
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from project_utils import *
from keras.regularizers import L1L2

In [ ]:
X, Y = data_preparing('data.csv', normalization = True)

train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size = 0.2)

In [ ]:
def create_model(n = 85, reg_input = L1L2(l1=0.0, l2=0.0), reg_bias = L1L2(l1=0.0, l2=0.0), drop_rate = 0.6):
    print('C R E A T E   M O D E L')
    model = Sequential()
    model.add(LSTM(n, return_sequences=True, kernel_regularizer=reg_input, bias_regularizer=reg_bias))
    model.add(Dropout(drop_rate))
    model.add(LSTM(n, return_sequences=True, kernel_regularizer=reg_input, bias_regularizer=reg_bias))
    model.add(Dropout(drop_rate))
    model.add(LSTM(n, return_sequences=True, kernel_regularizer=reg_input, bias_regularizer=reg_bias))
    model.add(Dropout(drop_rate))
    model.add(LSTM(n, return_sequences=False))
    model.add(Dense(1, activation = 'linear'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=RMSprop(lr = 0.001, clipvalue=0.3),
                  metrics=['acc', 'mean_absolute_error'])
    return model

model = KerasRegressor(build_fn=create_model, verbose=1, epochs = 50, batch_size = 6)

In [ ]:
# GRID SEARCH  - testing epochs, batch, optimizer, regularization#
batch_size = [5, 20]
epochs = [10, 50]
#optimizer = ['adam', 'rmsprop', 'adadelta', 'nadam']

param_grid = dict(batch_size = batch_size, epochs = epochs)
grid = GridSearchCV(cv = 3, estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)

# summarize results - grid_search 2
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
# define the grid search parameters - testing regularizers

reg_input = [L1L2(l1=0.0, l2=0.0), L1L2(l1=0.01, l2=0.0), L1L2(l1=0.0, l2=0.01), L1L2(l1=0.01, l2=0.01)]
reg_bias = [L1L2(l1=0.0, l2=0.0), L1L2(l1=0.01, l2=0.0), L1L2(l1=0.0, l2=0.01), L1L2(l1=0.01, l2=0.01)]
#reg_recurr = [L1L2(l1=0.0, l2=0.0), L1L2(l1=0.01, l2=0.0), L1L2(l1=0.0, l2=0.01), L1L2(l1=0.01, l2=0.01)]
#drop = [0.0, 0.2, 0.4, 0.6]

param_grid = dict(reg_input = reg_input, reg_bias = reg_bias)
grid = GridSearchCV(cv = 2, estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)

# summarize results - grid_search 1
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Define Callbacks
early_stopping = EarlyStopping(patience = 18, restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(patience = 15, verbose=1)
model_checkpoint = ModelCheckpoint('fas_mnist_1.h5', monitor = 'val_mean_absolute_error', verbose=1, save_best_only=True)

In [ ]:
# train the LSTM model - single training (one-fold)
model = create_model()
history = model.fit(train_X, train_y, validation_data = (val_X, val_y), epochs = 180, batch_size = 4, callbacks = 
                    [early_stopping, reduce_lr_on_plateau, model_checkpoint])
#evaluate model
score = model.evaluate()

#plot results 
plot_cv_history(history)

In [ ]:
# train the LSTM model - K-fold cross validation test

# define 6-fold cross validation test harness
splits = 6
kfold = KFold(n_splits=splits, shuffle = True)

# save CV results
cvscores = []
model_history = [] 

for train, test in kfold.split(X, Y):
      
    # model create and compile
    model = create_model()
    # model fit
    history = model.fit(X[train], Y[train], validation_data = (X[test], Y[test]), epochs=110, batch_size=4, verbose=1,
             callbacks = [early_stopping, reduce_lr_on_plateau, model_checkpoint])
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=1)
    print("%s: %.2f" % (model.metrics_names[2], scores[2]))
    cvscores.append(scores[2])
    model_history.append(history)
print("mean MAE: s%.2f (+/- %.2f)" % (np.mean(cvscores), np.std(cvscores)))

#plot results 
plot_cv_history(history, cv = splits)

# save history to Data Frame and csv file
historyDataFrame = pd.DataFrame()
for i in range(splits):
    historyDataFrame['Loss_{}'.format(i)] = pd.Series(model_history[i].history['loss'])
    historyDataFrame['val_Loss_{}'.format(i)] = pd.Series(model_history[i].history['val_loss'])
    historyDataFrame['Mae_{}'.format(i)] = pd.Series(model_history[i].history['mean_absolute_error'])
    historyDataFrame['val_Mae_{}'.format(i)] = pd.Series(model_history[i].history['val_mean_absolute_error'])
    
historyDataFrame.to_csv('model_history.csv')

In [ ]:
model.summary()